<a href="https://colab.research.google.com/github/CEFab/redes-neuronales-notebooks/blob/main/reconocimiento_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.5 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.8.0
    Uninstalling keras-3.8.0:
      Successfully uninstalled keras-3.8.0


# Clonamos el repositorio para obtener el dataset

In [2]:
!git clone https://github.com/joanby/deeplearning-az.git

Cloning into 'deeplearning-az'...
remote: Enumerating objects: 10295, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 10295 (delta 57), reused 50 (delta 39), pack-reused 10200 (from 1)
Receiving objects: 100% (10295/10295), 297.62 MiB | 32.65 MiB/s, done.
Resolving deltas: 100% (138/138), done.
Updating files: 100% (20243/20243), done.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Parte 1 - Construir el modelo de CNN

# Importar las librerías y paquetes

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Inicializar la CNN

In [6]:
classifier = Sequential()


# Paso 1 - Convolución

In [7]:
classifier.add(Conv2D(filters = 32,kernel_size = (3, 3),
                      input_shape = (64, 64, 3), activation = "relu"))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Paso 2 - Max Pooling

In [8]:
classifier.add(MaxPooling2D(pool_size = (2,2)))

# Una segunda capa de convolución y max pooling

In [9]:
classifier.add(Conv2D(filters = 32,kernel_size = (3, 3), activation = "relu"))

In [10]:
classifier.add(MaxPooling2D(pool_size = (2,2)))


# Paso 3 - Flattening

In [11]:
classifier.add(Flatten())

# Paso 4 - Full Connection

In [12]:
classifier.add(Dense(units = 128, activation = "relu"))
classifier.add(Dense(units = 1, activation = "sigmoid"))

# Compilar la CNN
# Como va a ser entrenada?

In [13]:
classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])


In [15]:
!ls -l "/content/deeplearning-az/datasets/Part 2 - Convolutional Neural Networks (CNN)/dataset/"

total 12
drwxr-xr-x 2 root root 4096 May 24 04:58 single_prediction
drwxr-xr-x 4 root root 4096 May 24 04:58 test_set
drwxr-xr-x 4 root root 4096 May 24 04:58 training_set


# Parte 2 - Ajustar la CNN a las imágenes para entrenar

In [17]:
from tensorflow.keras.preprocessing import image

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_dataset = train_datagen.flow_from_directory('/content/deeplearning-az/datasets/Part 2 - Convolutional Neural Networks (CNN)/dataset/training_set',
                                                    target_size=(64, 64),
                                                    batch_size=32,
                                                    class_mode='binary')

testing_dataset = test_datagen.flow_from_directory('/content/deeplearning-az/datasets/Part 2 - Convolutional Neural Networks (CNN)/dataset/test_set',
                                                target_size=(64, 64),
                                                batch_size=32,
                                                class_mode='binary')

classifier.fit(training_dataset,
                        steps_per_epoch=len(training_dataset),
                        epochs=25,
                        validation_data=testing_dataset,
                        validation_steps=len(testing_dataset))



Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 23s 91ms/step - accuracy: 0.7823 - loss: 0.4610 - val_accuracy: 0.7890 - val_loss: 0.4706
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 41s 91ms/step - accuracy: 0.7911 - loss: 0.4465 - val_accuracy: 0.7870 - val_loss: 0.4628
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 22s 87ms/step - accuracy: 0.8009 - loss: 0.4232 - val_accuracy: 0.7925 - val_loss: 0.4639
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 22s 89ms/step - accuracy: 0.8140 - loss: 0.4031 - val_accuracy: 0.8070 - val_loss: 0.4426
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 23s 91ms/step - accuracy: 0.8229 - loss: 0.3937 - val_accuracy: 0.7995 - val_loss: 0.4480
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 41s 91ms/step - accuracy: 0.8266 - loss: 0.3824 - val_accuracy: 0.7690 - val_loss: 0.4965
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 23s 91ms/step - accuracy: 0.8307 - loss: 0.3787 - val_accuracy: 0.7970 - val_loss: 0.4637
Epoch 

# Guardar modelo entrenado

In [18]:
# 1ra opcion: formato .h5
# Guardar el modelo en Google Drive
classifier.save('/content/drive/MyDrive/modelo_entrenado.h5')  # Formato HDF5

In [19]:
# 2da opcion: formato .keras
# Guardar el modelo en formato .keras (recomendado)
classifier.save('/content/drive/MyDrive/modelo_entrenado.keras')

# Parte 3 - Cómo hacer nuevas predicciones

1ra opcion: Con modelo recien entrenado

In [20]:
# 1ra opcion, modelo recien entrenado
import numpy as np
from tensorflow.keras.preprocessing import image  # Corrige la importación

# Cargar la imagen
test_image = image.load_img(
    '/content/deeplearning-az/datasets/Part 2 - Convolutional Neural Networks (CNN)/dataset/single_prediction/cat_or_dog_2.jpg',
    target_size=(64, 64)
)

# Preprocesamiento
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0) / 255.0  # Normaliza (¡importante!)

# Predicción
result = classifier.predict(test_image)

# Obtener las clases (asegúrate de que training_dataset está definido)
class_labels = list(training_dataset.class_indices.keys())  # Ej: ['cats', 'dogs']

# Determinar la clase
predicted_class = class_labels[int(result[0][0] > 0.5)]  # Umbral 0.5 para binario

# Mostrar resultado
print("Predicción:", predicted_class)
print("Confianza:", result[0][0] if predicted_class == 'dog' else 1 - result[0][0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 655ms/step
Predicción: cats
Confianza: 0.8156208


2da opcion: Con modelo guardado

In [21]:
from tensorflow.keras.models import load_model  # Importar load_model

# 1. Cargar el modelo guardado
modelo_cargado = load_model('/content/drive/MyDrive/modelo_entrenado.keras')  # Ruta correcta


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [24]:
# 2. Cargar y preprocesar la imagen (igual que antes)
test_image = image.load_img(
    '/content/deeplearning-az/datasets/Part 2 - Convolutional Neural Networks (CNN)/dataset/single_prediction/cat_or_dog_2.jpg',
    target_size=(64, 64)
)
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0) / 255.0  # Normalizar

# 3. Hacer la predicción con el modelo cargado
result = modelo_cargado.predict(test_image)  # Usar modelo_cargado, no classifier

# 4. Obtener clases y resultado (asegúrate de tener training_dataset definido)
class_labels = list(training_dataset.class_indices.keys())
predicted_class = class_labels[int(result[0][0] > 0.5)]
confianza = result[0][0] if predicted_class == 'dog' else 1 - result[0][0]

print("Predicción:", predicted_class)
print("Confianza:", confianza)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicción: cats
Confianza: 0.8156208
